# Parallelizing image convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-09-11 06:38:12,462	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 148.4 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-09-11 06:38:51,727	INFO worker.py:1783 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.35.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 85.7, speedup = 1.73


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 139.6, speedup = 1.06


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:42351' processes=4 threads=4, memory=15.61 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-09-11 06:42:41,631 - distributed.worker - ERROR - Compute Failed
Key:       fconv-0d82dbba2235983a0230720ffcf3c5d4
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.89271556,  1.89606605,  0.8007966 , -0.45852906],
       [ 0.12965298, -0.32687469, -0.38183771, -0.28769421],
       [ 0.34925568,  0.1857864 ,  0.60000333, -0.22573323],
       [ 0.15839486, -0.4687706 ,  1.74439704,  0.55911469]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:42:41,637 - distributed.worker - ERROR - Compute Failed
Key:       fconv-a36e3ad5b20ec66487deb65675a84da4
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-11 06:42:41,906 - distributed.worker - ERROR - Compute Failed
Key:       fconv-cf554e7475de6d49ee7189ecdf19412a
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.89271556,  1.89606605,  0.8007966 , -0.45852906],
       [ 0.12965298, -0.32687469, -0.38183771, -0.28769421],
       [ 0.34925568,  0.1857864 ,  0.60000333, -0.22573323],
       [ 0.15839486, -0.4687706 ,  1.74439704,  0.55911469]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:42:41,996 - distributed.worker - ERROR - Compute Failed
Key:       fconv-3c644bed215588412e3bd4fae42bf1e0
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-11 06:42:42,182 - distributed.worker - ERROR - Compute Failed
Key:       fconv-9751d3781fee1b2ba409d22ebf67b64f
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.9269997 ,  0.06211521, -0.66091152,  1.07334516],
       [-0.26268063, -0.88567147,  0.67068318, -0.98548818],
       [-0.65012489,  1.78492324,  0.20099093, -2.23471234],
       [-0.85650416, -0.44981666, -0.61272941, -0.64829695]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:42:42,272 - distributed.worker - ERROR - Compute Failed
Key:       fconv-eeb8cdda5840badde183edbe68ead217
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-11 06:42:42,455 - distributed.worker - ERROR - Compute Failed
Key:       fconv-5cfa36baff73f06e457842aa7b3f4b1e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.59290804, -0.24833328, -0.46364857, -0.32005034],
       [ 0.52320531, -0.01851771,  0.46425437, -1.93576608],
       [ 1.70774769,  0.15381107,  1.25584449,  0.81454805],
       [ 0.14928397,  0.630295  ,  0.1940378 ,  2.15333104]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:42:42,554 - distributed.worker - ERROR - Compute Failed
Key:       fconv-920c8b883be8dead9354816fe9d613d9
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-11 06:42:42,737 - distributed.worker - ERROR - Compute Failed
Key:       fconv-20535215ed014108a9fe220f80cbb29e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.57926214,  0.27771037, -0.0811993 ,  0.78187009],
       [-1.26982538,  0.0771931 , -1.80291206,  0.63811236],
       [-0.74795985, -2.7168081 , -1.6793085 ,  0.61281142],
       [ 1.73159755,  0.86748022,  1.05088433, -0.09651998]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:42:42,822 - distributed.worker - ERROR - Compute Failed
Key:       fconv-85cdcfae731dc2ede1a2d94d54240495
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-09-11 06:43:17,080 - distributed.worker - ERROR - Compute Failed
Key:       fconv-5326b1a0b0d8b95b212b8e0823c1dd0b
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.9269997 ,  0.06211521, -0.66091152,  1.07334516],
       [-0.26268063, -0.88567147,  0.67068318, -0.98548818],
       [-0.65012489,  1.78492324,  0.20099093, -2.23471234],
       [-0.85650416, -0.44981666, -0.61272941, -0.64829695]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:43:17,170 - distributed.worker - ERROR - Compute Failed
Key:       fconv-8301b4dda17b0bf3b38dfd99fb632a5d
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-11 06:43:17,362 - distributed.worker - ERROR - Compute Failed
Key:       fconv-70e0ba280c9f3761ae68036baddba6e2
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.59290804, -0.24833328, -0.46364857, -0.32005034],
       [ 0.52320531, -0.01851771,  0.46425437, -1.93576608],
       [ 1.70774769,  0.15381107,  1.25584449,  0.81454805],
       [ 0.14928397,  0.630295  ,  0.1940378 ,  2.15333104]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:43:17,452 - distributed.worker - ERROR - Compute Failed
Key:       fconv-ebf74feaa6beb6a18d97864b7efc6f6e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-09-11 06:43:17,635 - distributed.worker - ERROR - Compute Failed
Key:       fconv-c5133f4de0ae9660aeca2f539d1430f9
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.57926214,  0.27771037, -0.0811993 ,  0.78187009],
       [-1.26982538,  0.0771931 , -1.80291206,  0.63811236],
       [-0.74795985, -2.7168081 , -1.6793085 ,  0.61281142],
       [ 1.73159755,  0.86748022,  1.05088433, -0.09651998]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:43:17,723 - distributed.worker - ERROR - Compute Failed
Key:       fconv-3a8e9594529ce9504f0afcbc58b81298
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 37.0, speedup = 4.01


2024-09-11 06:43:17,910 - distributed.worker - ERROR - Compute Failed
Key:       fconv-32f9b0daf898858abf96b37fdfb91893
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.89271556,  1.89606605,  0.8007966 , -0.45852906],
       [ 0.12965298, -0.32687469, -0.38183771, -0.28769421],
       [ 0.34925568,  0.1857864 ,  0.60000333, -0.22573323],
       [ 0.15839486, -0.4687706 ,  1.74439704,  0.55911469]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-09-11 06:43:18,003 - distributed.worker - ERROR - Compute Failed
Key:       fconv-69bf6aa11db584c533b4e823c6963a32
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

In [16]:
client.close()

2024-09-11 06:43:18,053 - distributed.worker - ERROR - Compute Failed
Key:       fconv-df245a35ab1472f8db275c49d0b94ae0
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.59290804, -0.24833328, -0.46364857, -0.32005034],
       [ 0.52320531, -0.01851771,  0.46425437, -1.93576608],
       [ 1.70774769,  0.15381107,  1.25584449,  0.81454805],
       [ 0.14928397,  0.630295  ,  0.1940378 ,  2.15333104]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''



## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io